<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliares: Catherine Benavides y Consuelo Rojas
- Ayudante: Nicolás Ojeda, Eduardo Moya

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Tomás Guzmán
- Nombre de alumno 2: Fabián Palma


### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `github.com/tomasfabianMDS/LABS`

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00


# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/Lab 9"

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv(path + '/sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

<ipython-input-5-74c5e2720bed>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import joblib

In [7]:
def convert_capacity(value):
    if 'ml' in value:
        return float(value.replace('ml', ''))
    elif 'lt' in value:
        return float(value.replace('lt', '')) * 1000
    else:
        return np.nan

df['capacity'] = df['capacity'].apply(convert_capacity)

In [8]:
# Separar los datos en conjuntos de entrenamiento (70%), validación (20%) y prueba (10%)
train_size = 0.7
valid_size = 0.2
test_size = 0.1

# Dividir en train y temporal (valid + test)
X_train, X_temp, y_train, y_temp = train_test_split(df.drop('quantity', axis=1), df['quantity'], train_size=train_size, random_state=42)

# Calcular la proporción de validación respecto a los datos temporales
valid_size_temp = valid_size / (valid_size + test_size)

# Dividir los datos temporales en validación y prueba
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, train_size=valid_size_temp, random_state=42)

In [9]:
# FunctionTransformer para extraer día, mes y año
def extract_date_features(df):
    df = df.copy()
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df.drop(columns=['date'], inplace=True)
    return df

date_transformer = FunctionTransformer(extract_date_features)

In [10]:
# Selección de columnas categóricas y numéricas
categorical_features = ['city', 'shop', 'brand', 'container', 'day', 'month', 'year']
numeric_features = ['lat', 'long', 'pop', 'capacity', 'price']

# ColumnTransformer para procesar las columnas adecuadamente
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

In [11]:
# Pipeline con DummyRegressor
baseline_pipeline = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])

# Entrenar el pipeline
baseline_pipeline.fit(X_train, y_train)

# Predicciones y cálculo del MAE en el conjunto de validación
y_valid_pred = baseline_pipeline.predict(X_valid)
baseline_mae = mean_absolute_error(y_valid, y_valid_pred)

print(f'MAE del DummyRegressor: {baseline_mae}')

MAE del DummyRegressor: 13298.497767341096


In [12]:
# Pipeline con XGBRegressor
xgb_pipeline = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor(objective='reg:squarederror'))
])

# Entrenar el pipeline
xgb_pipeline.fit(X_train, y_train)

# Predicciones y cálculo del MAE en el conjunto de validación
y_valid_pred_xgb = xgb_pipeline.predict(X_valid)
xgb_mae = mean_absolute_error(y_valid, y_valid_pred_xgb)

print(f'MAE del XGBRegressor: {xgb_mae}')

MAE del XGBRegressor: 2426.9676805952904


In [13]:
# Guardar los modelos en archivos .pkl
joblib.dump(baseline_pipeline, 'baseline_pipeline.pkl')
joblib.dump(xgb_pipeline, 'xgb_pipeline.pkl')

['xgb_pipeline.pkl']

En este caso, el XGBRegressor ha mostrado un MAE significativamente menor comparado con el DummyRegressor, lo que indica que está haciendo un mejor trabajo en la predicción de la demanda.

## 2. Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. Hint: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación.

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?




In [14]:
# Define a list containing the constraint dictionary
constraints = [
    {
        'feature': 'price',
        'direction': 'decreasing',
        'threshold': 0
    }
]

# Ensure 'feature_names' attribute is available
xgb_pipeline_constrained = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True))  # Enable categorical support
])

# Fit the pipeline with the corrected constraint format
xgb_pipeline_constrained.fit(X_train, y_train)

Pipeline(steps=[('date_transformer',
                 FunctionTransformer(func=<function extract_date_features at 0x7a5373b755a0>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['lat', 'long', 'pop',
                                                   'capacity', 'price']),
                                                 ('cat', OneHotEncoder(),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'day', 'month',
                                                   'year'])])),
                ('regressor',
                 XGBRegressor(base_score=None,...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [15]:
# Predicciones y cálculo del MAE en el conjunto de validación
y_valid_pred_xgb_monotonic = xgb_pipeline_constrained.predict(X_valid)
xgb_mae_monotonic = mean_absolute_error(y_valid, y_valid_pred_xgb_monotonic)

print(f'MAE del XGBRegressor con restricción monotónica: {xgb_mae_monotonic}')

MAE del XGBRegressor con restricción monotónica: 2426.9676805952904


In [16]:
# Comparación de MAE
print(f'MAE del XGBRegressor sin restricción: {xgb_mae}')
print(f'MAE del XGBRegressor con restricción monotónica: {xgb_mae_monotonic}')

MAE del XGBRegressor sin restricción: 2426.9676805952904
MAE del XGBRegressor con restricción monotónica: 2426.9676805952904


In [17]:
# Guardar el modelo con la restricción monotónica en un archivo .pkl
joblib.dump(xgb_pipeline_constrained, 'xgb_pipeline_constrained.pkl')

['xgb_pipeline_constrained.pkl']

En teoría, deberíamos haber observado una disminución en el MAE del modelo XGBoost con restricción monotónica en comparación con el modelo XGBoost sin restricciones. El hecho de que el MAE haya sido el mismo en ambos casos sugiere que la restricción monotónica no tuvo un efecto significativo en la capacidad del modelo para capturar la relación entre "price" y "quantity". Esto podría indicar que la relación entre estas variables en los datos no sigue una tendencia monotónica clara, o que la implementación de la restricción no fue efectiva en este caso particular.

## 3. Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [18]:
# Define the constrained XGBoost model with initial parameters
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True,
                          learning_rate=0.05, n_estimators=100, max_depth=5,
                          max_leaves=10, min_child_weight=1, reg_alpha=0.3, reg_lambda=0.1)

constrained_pipeline = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', xgb_model)
])

In [73]:
import time
import optuna

# Define Optuna objective function
def objective(trial):
    # Set hyperparameters based on trial values
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True,
                              learning_rate=0.05, n_estimators=100, max_depth=5,
                              max_leaves=10, min_child_weight=1, reg_alpha=0.3, reg_lambda=0.1)
# Define Optuna objective function with prunning callback
    # Set hyperparameters based on trial values
    parametros = {
    'learning_rate' : trial.suggest_float('learning_rate', 0.001, 0.1),
    'n_estimators' : trial.suggest_int('n_estimators', 50, 1000),
    'max_depth' : trial.suggest_int('max_depth', 3, 10),
    'max_leaves' : trial.suggest_int('max_leaves', 0, 100),
    'min_child_weight' : trial.suggest_int('min_child_weight', 1, 5),
    'reg_alpha' : trial.suggest_float('reg_alpha', 0, 1),
    'reg_lambda' : trial.suggest_float('reg_lambda', 0, 1)
    }
    min_frequency = trial.suggest_float('min_frequency', 0.0, 0.1)
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(min_frequency = min_frequency), categorical_features)
    ]
)
    constrained_pipeline = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', xgb.XGBRegressor(**parametros))
    ])
    # Train the constrained XGBoost model
    constrained_pipeline.fit(X_train, y_train)

    # Make predictions on validation set
    y_valid_pred = constrained_pipeline.predict(X_valid)

    # Evaluate MAE on validation set
    mae = mean_absolute_error(y_valid, y_valid_pred)

    return mae

# Set Optuna study
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, timeout=300)

# Print best trial and parameters
best_trial = study.best_trial
print(f"Number of trials: {len(study.trials)}")
print(f"Best MAE: {best_trial.value}")
print("Best parameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")


Number of trials: 114
Best MAE: 1930.5552733720028
Best parameters:
  learning_rate: 0.06616713585245172
  n_estimators: 853
  max_depth: 10
  max_leaves: 94
  min_child_weight: 4
  reg_alpha: 0.9380829440646753
  reg_lambda: 0.7858335843187562
  min_frequency: 0.029517996019777226


In [82]:
import joblib
best_params = best_trial.params
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True, **best_params)
constrained_pipeline = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', xgb_model)
])
constrained_pipeline.fit(X_train, y_train)

joblib.dump(constrained_pipeline, 'constrained_xgb_model_optimized.pkl')

['constrained_xgb_model_optimized.pkl']

Explicación y Resultados
- **learning_rate**: Controla el tamaño de los pasos dados hacia el mínimo durante el entrenamiento. Valores bajos pueden hacer que el entrenamiento sea más robusto, pero más lento.
- **n_estimators**: Aumentar este número suele mejorar la precisión, pero también puede llevar al sobreajuste y aumentar el tiempo de entrenamiento.
- **max_depth** y **max_leaves**: Controlan la complejidad del modelo. Profundidades o números de hojas muy grandes pueden llevar al sobreajuste.
- **min_child_weight**: Evita que el modelo ajuste demasiado a pequeños grupos de datos.
- **reg_alpha** y **reg_lambda**: Penalizan la complejidad del modelo para evitar el sobreajuste.

El MAE mejora con respecto a las iteraciones anteriores. Esto se explica comprendiendo que la optimización bayesiana explora el espacio de hiperparámetros de manera más eficiente y puede encontrar combinaciones óptimas que no son evidentes con una búsqueda manual.

## 4. Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

Prunning, o poda, es una técnica utilizada en el entrenamiento de modelos de machine learning para evitar el sobreajuste y mejorar la generalización del modelo. Consiste en detener el entrenamiento de un modelo cuando se detecta que no se está obteniendo ninguna mejora significativa en la métrica de interés en un período de tiempo determinado. Esto ayuda a evitar que el modelo continúe entrenándose demasiado y se ajuste demasiado a los datos de entrenamiento, lo que podría llevar a un rendimiento deficiente en datos no vistos.

La implementación de prunning utilizando optuna.integration.XGBoostPruningCallback permite monitorear el rendimiento del modelo durante el entrenamiento y detenerlo si no cumple con ciertos criterios de mejora, como por ejemplo, si la métrica de interés no mejora durante un número determinado de iteraciones consecutivas en el conjunto de validación.

In [20]:
!pip install optuna-integration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 2.3 MB/s eta 0:00:00


In [46]:
def pipeline_fit_with_eval_set(pipeline, X_train, y_train, X_test, y_test, fit_params={}):
    """
    Fit a scikit-learn pipeline with eval_set support.

    Parameters:
    - pipeline: The scikit-learn pipeline.
    - X_train: Training data.
    - y_train: Training labels.
    - X_test: Test data.
    - y_test: Test labels.
    - fit_params: Additional fit parameters.
    - pipeline_model_step_name: Name of the model step in the pipeline.

    Usage:
    pipeline_fit_with_eval_set(my_pipeline, X_train, y_train, X_test, y_test, fit_params={'eval_metric': 'logloss'})
    """
    # Step 1: Extract Preprocessors
    pipeline_preprocessors = Pipeline(pipeline.steps[:-1])

    # Step 2: Fit preprocessors and Transform Training Data
    # Make sure not to use any test data for the fit step
    X_train_transformed = pipeline_preprocessors.fit_transform(X_train)

    # Step 3: Transform Test Data
    X_test_transformed = pipeline_preprocessors.transform(X_test)

    # Step 4: Prepare Eval Set
    fit_params["eval_set"] = [(X_test_transformed, y_test)]

    # Step 5: Extract Model and Fit
    model = pipeline.steps[-1][1]
    model.fit(X_train_transformed, y_train, **fit_params)

In [63]:
from optuna.integration.xgboost import XGBoostPruningCallback
import optuna.logging

optuna.logging.set_verbosity(optuna.logging.WARNING)
def objective(trial):
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True,
                              learning_rate=0.05, n_estimators=100, max_depth=5,
                              max_leaves=10, min_child_weight=1, reg_alpha=0.3, reg_lambda=0.1)


    parametros = {
    'learning_rate' : trial.suggest_float('learning_rate', 0.001, 0.1),
    'n_estimators' : trial.suggest_int('n_estimators', 50, 1000),
    'max_depth' : trial.suggest_int('max_depth', 3, 10),
    'max_leaves' : trial.suggest_int('max_leaves', 0, 100),
    'min_child_weight' : trial.suggest_int('min_child_weight', 1, 5),
    'reg_alpha' : trial.suggest_float('reg_alpha', 0, 1),
    'reg_lambda' : trial.suggest_float('reg_lambda', 0, 1)
    }
    min_frequency = trial.suggest_float('min_frequency', 0.0, 0.1)
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(min_frequency = min_frequency), categorical_features)
    ]
)
    constrained_pipeline = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', xgb.XGBRegressor(**parametros))
    ])
    # Update XGBoost model parameters
    # xgb_model.set_params(learning_rate=learning_rate, n_estimators=n_estimators,
    #                      max_depth=max_depth, max_leaves=max_leaves,
    #                      min_child_weight=min_child_weight, reg_alpha=reg_alpha,
    #                      reg_lambda=reg_lambda)

    pipeline_fit_with_eval_set(constrained_pipeline, X_train, y_train, X_valid, y_valid, fit_params = {'callbacks': [XGBoostPruningCallback(trial, "validation_0-rmse")]})
    y_valid_pred = constrained_pipeline.predict(X_valid)

    mae = mean_absolute_error(y_valid, y_valid_pred)

    return mae

# Setear Optuna study
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, timeout=5 * 60)  # 5 minutes

# Print best trial and parameters
best_trial = study.best_trial
print(f"Number of trials: {len(study.trials)}")
print(f"Best MAE: {best_trial.value}")
print("Best parameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

[0]	validation_0-rmse:17288.50739
[1]	validation_0-rmse:16957.51970


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning:

`callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.



[2]	validation_0-rmse:16641.01852
[3]	validation_0-rmse:16344.54613
[4]	validation_0-rmse:16049.41963
[5]	validation_0-rmse:15756.32457
[6]	validation_0-rmse:15491.75286
[7]	validation_0-rmse:15225.44977
[8]	validation_0-rmse:14973.48166
[9]	validation_0-rmse:14720.12957
[10]	validation_0-rmse:14474.23835
[11]	validation_0-rmse:14246.45946
[12]	validation_0-rmse:14021.99916
[13]	validation_0-rmse:13800.56117
[14]	validation_0-rmse:13599.44921
[15]	validation_0-rmse:13396.31807
[16]	validation_0-rmse:13204.74430
[17]	validation_0-rmse:13005.65336
[18]	validation_0-rmse:12821.99162
[19]	validation_0-rmse:12646.37844
[20]	validation_0-rmse:12468.39583
[21]	validation_0-rmse:12291.34623
[22]	validation_0-rmse:12106.72925
[23]	validation_0-rmse:11952.23929
[24]	validation_0-rmse:11789.71720
[25]	validation_0-rmse:11634.16191
[26]	validation_0-rmse:11478.41822
[27]	validation_0-rmse:11328.42940
[28]	validation_0-rmse:11183.24625
[29]	validation_0-rmse:11041.12736
[30]	validation_0-rmse:10901

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning:

`callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.



[17]	validation_0-rmse:10146.71451
[18]	validation_0-rmse:9981.82968
[19]	validation_0-rmse:9856.52744
[20]	validation_0-rmse:9732.01906
[21]	validation_0-rmse:9604.22205
[22]	validation_0-rmse:9491.21618
[23]	validation_0-rmse:9396.05791
[24]	validation_0-rmse:9290.47097
[25]	validation_0-rmse:9207.70888
[26]	validation_0-rmse:9136.56002
[27]	validation_0-rmse:9059.46801
[28]	validation_0-rmse:9004.41409
[29]	validation_0-rmse:8921.90693
[30]	validation_0-rmse:8868.82302
[31]	validation_0-rmse:8826.36567
[32]	validation_0-rmse:8783.40080
[33]	validation_0-rmse:8736.04656
[34]	validation_0-rmse:8695.63939
[35]	validation_0-rmse:8658.84921
[36]	validation_0-rmse:8627.57603
[37]	validation_0-rmse:8594.62691
[38]	validation_0-rmse:8577.34731
[39]	validation_0-rmse:8549.84920
[40]	validation_0-rmse:8530.52956
[41]	validation_0-rmse:8499.17941
[42]	validation_0-rmse:8481.56777
[43]	validation_0-rmse:8465.43586
[44]	validation_0-rmse:8433.77075
[45]	validation_0-rmse:8415.42307
[46]	validati

...

[2]	validation_0-rmse:14326.90961
[3]	validation_0-rmse:13472.35537
[4]	validation_0-rmse:12669.01776
[5]	validation_0-rmse:11974.90989
[6]	validation_0-rmse:11364.00384
[7]	validation_0-rmse:10789.24741
[8]	validation_0-rmse:10241.06698
[9]	validation_0-rmse:9769.10748
[10]	validation_0-rmse:9356.96787
[11]	validation_0-rmse:8952.53575
[12]	validation_0-rmse:8570.37703
[13]	validation_0-rmse:8249.41496
[14]	validation_0-rmse:7950.20172
[15]	validation_0-rmse:7653.99547
[16]	validation_0-rmse:7375.42982
[17]	validation_0-rmse:7125.79817
[18]	validation_0-rmse:6906.86839
[19]	validation_0-rmse:6718.28869
[20]	validation_0-rmse:6502.66705
[21]	validation_0-rmse:6339.65243
[22]	validation_0-rmse:6150.01923
[23]	validation_0-rmse:5980.90337
[24]	validation_0-rmse:5853.11026
[25]	validation_0-rmse:5733.91291
[26]	validation_0-rmse:5609.67341
[27]	validation_0-rmse:5500.33389
[28]	validation_0-rmse:5377.57795
[29]	validation_0-rmse:5275.46304
[30]	validation_0-rmse:5197.73621
[31]	validation

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning:

`callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.



[8]	validation_0-rmse:10241.08590
[9]	validation_0-rmse:9768.88768
[10]	validation_0-rmse:9356.72809
[11]	validation_0-rmse:8952.02799
[12]	validation_0-rmse:8567.64995
[13]	validation_0-rmse:8235.15154
[14]	validation_0-rmse:7940.99714
[15]	validation_0-rmse:7659.90593
[16]	validation_0-rmse:7394.52541
[17]	validation_0-rmse:7146.71525
[18]	validation_0-rmse:6895.73889
[19]	validation_0-rmse:6674.63762
[20]	validation_0-rmse:6493.59078
[21]	validation_0-rmse:6338.27783
[22]	validation_0-rmse:6155.24762
[23]	validation_0-rmse:6014.50967
[24]	validation_0-rmse:5863.71801
[25]	validation_0-rmse:5737.45140
[26]	validation_0-rmse:5598.44175
[27]	validation_0-rmse:5487.33954
[28]	validation_0-rmse:5358.19973
[29]	validation_0-rmse:5246.42467
[30]	validation_0-rmse:5169.02808
[31]	validation_0-rmse:5061.96215
[32]	validation_0-rmse:4966.42662
[33]	validation_0-rmse:4900.38910
[34]	validation_0-rmse:4825.64588
[35]	validation_0-rmse:4757.04500
[36]	validation_0-rmse:4692.83220
[37]	validation

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning:

`callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.



[8]	validation_0-rmse:10278.46943
[9]	validation_0-rmse:9825.67514
[10]	validation_0-rmse:9398.16051
[11]	validation_0-rmse:8957.14362
[12]	validation_0-rmse:8594.54552
[13]	validation_0-rmse:8233.95760
[14]	validation_0-rmse:7925.32436
[15]	validation_0-rmse:7632.50951
[16]	validation_0-rmse:7372.15410
[17]	validation_0-rmse:7125.59951
[18]	validation_0-rmse:6873.97093
[19]	validation_0-rmse:6636.18241
[20]	validation_0-rmse:6444.34275
[21]	validation_0-rmse:6273.04629
[22]	validation_0-rmse:6118.36848
[23]	validation_0-rmse:5957.44822
[24]	validation_0-rmse:5804.10635
[25]	validation_0-rmse:5650.66921
[26]	validation_0-rmse:5524.49399
[27]	validation_0-rmse:5413.26282
[28]	validation_0-rmse:5325.79571
[29]	validation_0-rmse:5235.54781
[30]	validation_0-rmse:5152.57958
[31]	validation_0-rmse:5069.18420
[32]	validation_0-rmse:4975.37082
[33]	validation_0-rmse:4909.02448
[34]	validation_0-rmse:4822.67080
[35]	validation_0-rmse:4774.20415
[36]	validation_0-rmse:4725.02770
[37]	validation

In [ ]:
import joblib
best_params = best_trial.params
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True, **best_params)
constrained_pipeline = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', xgb_model)
])
constrained_pipeline.fit(X_train, y_train)

joblib.dump(constrained_pipeline, 'constrained_xgb_model_prunned.pkl')

## 5. Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización
2. Gráfico de coordenadas paralelas
3. Gráfico de importancia de hiperparámetros

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados?
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [64]:
# Inserte su código acá
import optuna

from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

In [70]:
plot_optimization_history(study)


Desde el trial 8 se empiezan a ver mejoras notables.

In [71]:
plot_parallel_coordinate(study)


La dispersión de las líneas en el gráfico indica la importancia relativa de cada hiperparámetro en la determinación del rendimiento del modelo. También se pueden identificar interacciones entre los hiperparámetros al observar cómo las líneas se comportan en relación con diferentes combinaciones, pues cuando las líneas de datos cruzan o convergen consistentemente en puntos específicos en múltiples ejes, indica una posible correlación entre las variables. Por el contrario, los valores atípicos, es decir, los puntos de datos que se desvían significativamente de los patrones esperados, destacan como líneas que divergen notablemente de la tendencia general.

In [72]:
plot_param_importances(study)


el top 3 de hiperparametros es: reg alpha, max_leaves y n_estimators

## 6. Síntesis de resultados (0.3)

Finalmente:

1. Genere una tabla resumen del MAE obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning.
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento?
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE.
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

| Modelo                                | MAE        |
|---------------------------------------|------------|
| DummyRegressor                        | 13298.50   |
| XGBRegressor                          | 2426.97    |
| XGBRegressor con restricción monotónica | 2426.97 |
| Best MAE optimización de parámetros   | 1930.56    |
| Best MAE prunners                     | 1979.52    |

Como puede verse en la tabla anterior, el modelo que presenta mejor rendimiento es el de XGB regressor con optimización de parámetros con Optuna.

In [80]:

loaded_pipeline = joblib.load('constrained_xgb_model_optimized.pkl')

y_new_pred = loaded_pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_new_pred)


In [81]:
mae

1942.9160650422043

En efecto, ocurren diferencias entre las métricas de validación y test. Esto no es extraño y puede explicarse por razones como la varianza en los datos, pues los conjuntos de validación y prueba son diferentes, lo que puede implicar variaciones naturales en las métricas. También puede deberse a un posible sobreajuste en el cual la optimización de hiperparámetros lleva al modelo ajustado específicamente para el conjunto de validación, mientras no generaliza tan bien en el conjunto de testeo.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>